## Getting Started

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.features import FeatureSet

In [37]:
gis = GIS(api_key="AAPK6ee8947e8f9549d0ada59f3e054f57e2fs3A0lQeIqocbpFkvx1hDM0cgFXyKXef3hmZNzOEwh79qVXTWqLFIbVAUjpGlPQ7")

## StudyAreas

In [8]:
# Input pharmacy locations by address
df = pd.read_csv("data/cvs_los_angeles.csv")
df

,Business Name,Address,City,State,ZIP
0,CVS,11051 Victory Blvd,North Hollywood,CA,91606
1,CVS,864 Swarthmore Ave,Pacific Palisades,CA,90272
2,CVS,15614 Whittwood Ln,Whittier,CA,90603
3,CVS,1025 E Adams Blvd,Los Angeles,CA,90011
4,CVS,4235 Pacific Coast Hwy,Torrance,CA,90505
...,...,...,...,...,...
489,CVS,25601 Jeronimo Rd,Mission Viejo,CA,92691
490,CVS,4040 E Chapman Ave,Orange,CA,92869
491,CVS,855 E Birch St,Brea,CA,92821
492,CVS,8601 S Sepulveda Blvd,Los Angeles,CA,90045


## Enriching Areas

In [15]:
# Defines the 5-min drive time analysis areas around each location
cvs_areas = [{"address":x, "areaType": "DriveTimeBuffer","bufferRadii":[5], "bufferUnits": "Minutes"} for x in df.to_dict('records')]
cvs_areas[0]

{'address': {'Business Name': 'CVS',
  'Address': '11051 Victory Blvd',
  'City': 'North Hollywood',
  'State': 'CA',
  'ZIP': 91606},
 'areaType': 'DriveTimeBuffer',
 'bufferRadii': [5],
 'bufferUnits': 'Minutes'}

### Find in **Data Browser**:

Site arcgis.com -> BA Web App -> Help -> Data documentation -> Data -> **Data Browser** -> find by "seniors"
-> 2021 Seniors (Age 65+) -> choose it -> Copy (JSON) -> paste in code this **Attribute IDs**: `["AtRisk.SENIORS_CY"]`

Paste:

In [22]:
# Drive time service areas are being generated dynamically for lots of locations
# (lots of processing required)
cvs_dt_5min = enrich(cvs_areas, analysis_variables=["AtRisk.SENIORS_CY"])

In [26]:
# We can see below
#   areaType: DriveTimeBuffer
#   bufferUnits: Minutes
#   bufferRadii: 5
#   aggregationMethod: BlockApportionment:US.BlockGroups
#   SENIORS_CY: <we can see>
cvs_dt_5min.head(n=3)

,ID,OBJECTID,sourceCountry,X,Y,areaType,bufferUnits,bufferUnitsAlias,bufferRadii,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,SENIORS_CY,SHAPE
0,0,1,US,-118.371276,34.188669,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,9109,"{""rings"": [[[-118.36978957234149, 34.219339094..."
1,1,2,US,-118.525640,34.046389,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,3223,"{""rings"": [[[-118.54248989609434, 34.060338017..."
2,2,3,US,-117.996562,33.942616,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,6150,"{""rings"": [[[-118.00170656811636, 33.961972943..."


In [31]:
# Let's map enrichment areas
cvs_map = gis.map('Ontario, CA, USA', zoomlevel=9)
cvs_map

MapView(layout=Layout(height='400px', width='100%'))

In [32]:
# Change basemap
cvs_map.basemap = 'dark-gray'

In [33]:
# Add layer
cvs_map.add_layer(
    FeatureSet.from_dataframe(cvs_dt_5min),
    {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5}
)

As a result we can see senior populations around our **5-min drive time** analysis areas around the pharmacies (*darker = larger population counts*).

## Data Analysis

In [35]:
# Sort by seniors amount
cvs_dt_5min = cvs_dt_5min.sort_values(by=['SENIORS_CY'], ascending=False)

# Take 10 most populated
best_csv = cvs_dt_5min[:10]
best_csv

,ID,OBJECTID,sourceCountry,X,Y,areaType,bufferUnits,bufferUnitsAlias,bufferRadii,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,SENIORS_CY,SHAPE
57,57,58,US,-118.286440,34.063251,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,20203,"{""rings"": [[[-118.28692036736885, 34.088634818..."
84,84,85,US,-118.258240,34.147581,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,17970,"{""rings"": [[[-118.25570405317951, 34.169931979..."
52,52,53,US,-118.258587,34.142978,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,17871,"{""rings"": [[[-118.29904756711058, 34.167686202..."
77,77,78,US,-118.099652,33.887662,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,17248,"{""rings"": [[[-118.10096995336235, 33.933676140..."
76,76,77,US,-118.061981,33.846908,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,17155,"{""rings"": [[[-118.06391461677569, 33.877756270..."
38,38,39,US,-118.109699,34.079314,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,16953,"{""rings"": [[[-118.10209284207761, 34.103456952..."
43,43,44,US,-117.992705,33.825497,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,16919,"{""rings"": [[[-117.99384634621127, 33.862709557..."
19,19,20,US,-118.052536,34.109551,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,16866,"{""rings"": [[[-118.05021537157494, 34.146126734..."
48,48,49,US,-117.989419,33.685158,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,16492,"{""rings"": [[[-117.98913021163087, 33.717856881..."
8,8,9,US,-118.027219,33.833707,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups;PointsLayer:...,2.191,2.576,1,15877,"{""rings"": [[[-118.02865590357078, 33.865629068..."


In [38]:
# Map areas
best_cvs_map = gis.map('Ontario, CA, USA', zoomlevel=10)
best_cvs_map

MapView(layout=Layout(height='400px', width='100%'))

In [39]:
# Change basemap
best_cvs_map.basemap = 'dark-gray'

In [40]:
# Add layer
best_cvs_map.add_layer(
    FeatureSet.from_dataframe(best_csv.copy()),
    {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5}
)

### Result:
- We can see top candidates on the map above.
- We can click on this areas for more detailed information.